In [1]:
import numpy as np
import torch.nn as nn

In [ ]:
class yolo_loss(nn.Module):
    def __init__(self):
        super(yolo_loss, self).__init__()

    def forward(self, 
                x_pred, x_actual, y_pred, y_actual,
                w_pred, w_actual, h_pred, h_actual,
                is_obj_pred, is_obj_actual,
                no_obj_pred, no_obj_actual, 
                class_pred, class_actual):
        ## constants
        lambda_coord = None
        lambda_no_obj = None

        obj_identity = None
        no_obj_identity = None

        ## bbox coordinate loss
        bbox_coord_norm = np.power((x_pred - x_actual), 2) + np.power((y_pred - y_actual), 2)
        bbox_coord_loss = lambda_coord*np.sum(np.sum(np.multiply(obj_identity, bbox_coord_norm)))

        ## bbox dimension loss
        bbox_dim_norm = np.power((np.sqrt(w_pred) - np.sqrt(w_actual)), 2) + np.power((np.sqrt(h_pred) - np.sqrt(h_actual)), 2)
        bbox_dim_loss = lambda_coord*np.sum(np.sum(np.multiply(obj_identity, bbox_dim_norm)))

        ## objectness loss
        obj_norm = np.power((is_obj_actual - is_obj_pred),2)
        obj_loss = np.sum(np.sum(np.multiply(obj_identity, obj_norm)))

        ## objectless loss
        no_obj_norm = np.power((no_obj_actual - no_obj_pred),2)
        no_obj_loss = lambda_no_obj*np.sum(np.sum(np.multiply(no_obj_identity, no_obj_norm)))

        ## class regression loss
        class_reg_norm = np.power(class_pred - class_actual, 2)
        class_reg_loss = np.sum(obj_identity*np.sum(class_reg_norm))
        
        ## combine for final loss
        loss = bbox_coord_loss + bbox_dim_loss + obj_loss + no_obj_loss + class_reg_loss

        return loss